#TUTS: d3-less basic chloropleth map using Python (Viz Tuts: 1 of 6)
##How to Make a US County Thematic Map Using Free Tools
##From: [FlowingData](http://flowingdata.com/2009/11/12/how-to-make-a-us-county-thematic-map-using-free-tools/)
##Downloaded: 4-May-2014

##0. PRELIMINARIES

In [1]:
import csv
from BeautifulSoup import BeautifulSoup

ImportError: No module named BeautifulSoup

In [ ]:
unemployment = {}

reader = csv.reader(open('processed_data/unemployment09.csv'), delimiter=',')
for row in reader:
    try:
        full_fips = row[1] + row[2]
        rate = float(row[8].strip())
        unemployment[full_fips] = rate
    except:
        pass

In [ ]:
#head
unemployment.iteritems().next()

Luckily, we don't have to start from scratch. We can get a blank USA counties map from Wikimedia Commons. We want the SVG one. Download the SVG file onto your computer and save it as counties.svg.

In [ ]:
#Load the SVG map
svg = open('processed_data/counties.svg', 'r').read()

#Load into BeautifulSoup
soup = BeautifulSoup(svg, selfClosingTags=['defs','sodipodi:namedview'])

In [ ]:
# soup.prettify()

Each path is a county. The long run of numbers are the coordinates for the county's boundary lines. We're not going to fuss with those numbers.

We only care about the beginning and very end of the path tag. We're going to change the style attribute, namely fill color. We want the darkness of fill to correspond to the unemployment rate in each given county.

We could change each one manually, but there are over 3,000 counties. That would take too long. Instead we'll use Beautiful Soup, an XML parsing Python library, to change colors accordingly.

Each path also has an id, which is actually something called a FIPS code. FIPS stands for Federal Information Processing Standard. Every county has a unique FIPS code, and it's how we are going to associate each path with our unemployment data.

##Step 1. Find all the counties in the SVG

Beautiful Soup has a nifty findAll() function that we can use to find all the counties in our SVG file.  All paths are stored in the paths array.

In [ ]:
#Find counties
paths = soup.findAll('path')

##Step 2. Decide what colors to use for map

There are plenty of color schemes to choose from, but if you don't want to think about it, give the ColorBrewer a whirl. It's a tool to help you decide your colors. For this particular map, I chose the PurpleRed scheme with six data classes.  In the bottom, left-hand corner, are our color codes. Select the hexadecimal option (HEX), and then create an array of those hexadecimal colors.

In [ ]:
# Map colors
colors = ["#F1EEF6", "#D4B9DA", "#C994C7", "#DF65B0", "#DD1C77", "#980043"]

##Step 3. Prepare style for paths

We're getting close to the climax. Like I said earlier, we're going to change the style attribute for each path in the SVG. We're just interested in fill color, but to make things easier we're going to replace the entire style instead of parsing to replace only the color.  Everything is the same as the previous style except we moved fill to the end and left the value blank. We're going to fill that in just a second. We also changed stroke to #FFFFFF to make county borders white. We didn't have to leave that value blank, because we want all borders to be white while fill depends on unemployment rate.

In [ ]:
# County style
path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start:none;stroke-linejoin:bevel;fill:'

##Step 4. Change color of counties

We're ready to change colors now! Loop through all the paths, find the unemployment rate from the unemployment dictionary, and then select color class accordingly. Notice the if statement. I don't want to change the style of state lines or the line that separates Hawaii and Alaska from the rest of the states.

I also hard-coded the conditions to decide the color class because I knew beforehand what the distribution is like. If however, you didn't know the distribution, you could use something like this: float(len(colors)-1) * float(rate - min_value) / float(max_value - min_value).

In [ ]:
# Color the counties based on unemployment rate
for p in paths:
     
    if p['id'] not in ["State_Lines", "separator"]:
        # pass
        try:
            rate = unemployment[p['id']]
        except:
            continue
             
        if rate > 10:
            color_class = 5
        elif rate > 8:
            color_class = 4
        elif rate > 6:
            color_class = 3
        elif rate > 4:
            color_class = 2
        elif rate > 2:
            color_class = 1
        else:
            color_class = 0
 
        color = colors[color_class]
        p['style'] = path_style + color

In [ ]:
from IPython.display import SVG
# SVG(filename='processed_data/counties.svg')
SVG(data=soup.prettify())

#TUTS: d3 chloropleth map with legend and interactive tooltip (Viz Tuts: 2 of 6)
##Making a Simple Interactive Map Prototype with D3…For Total Beginners Who are Totally Impatient
##From: [SUFFEN.US](https://suffenus.wordpress.com/2014/01/07/making-interactive-maps-with-d3-for-total-beginners/)
##Downloaded: 4-May-2014

In [1]:
%%html

<div id="interactive-map"></div>

<style>
    path {
     stroke:white;
     stroke-width: 1px;
    }
    
    .legend {
     font-size: 12px;
    }

    .city {
     font: 10px sans-serif;
     font-weight: bold;
    }
    
    div.tooltip {
     position: absolute;
     text-align: center;
     width: 150px;
     height: 25px;
     padding: 2px;
     font-size: 10px;
     background: #FFFFE0;
     border: 1px;
     border-radius: 8px;
     pointer-events: none;
    }

</style>

In [2]:
%%javascript

require.config({paths: {d3: "http://d3js.org/d3.v3.min", queue:"http://d3js.org/queue.v1.min", topojson:"http://d3js.org/topojson.v1.min"}});

require(["d3", "queue", "topojson"], function(d3, queue, topojson) {
    
    var width = 960,
        height = 500;
    
    var color_domain = [500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000]
    var ext_color_domain = [0, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000]
    var legend_labels = ["< 500", "500+", "1000+", "1500+", "2000+", "2500+", "3000+", "3500+", "4000+", "4500+", "5000+", "5500+", "6000+"]
    var color = d3.scale.threshold()
    .domain(color_domain)
    .range(["#dcdcdc", "#d0d6cd", "#bdc9be", "#aabdaf", "#97b0a0", "#84a491", "#719782", "#5e8b73", "#4b7e64", "#387255", "#256546", "#125937", "#004d28"]);

    var div = d3.select("#interactive-map").append("div")
     .attr("class", "tooltip")
     .style("opacity", 0);
    
    var svg = d3.select("#interactive-map").append("svg")
     .attr("width", width)
     .attr("height", height);
     var path = d3.geo.path()
     
    queue()
     .defer(d3.json, "processed_data/usa.json")
     .defer(d3.csv, "processed_data/data.csv")
     .await(ready);
    
    function ready(error, us, data) {
        var pairRateWithId = {};
        var pairNameWithId = {};

        data.forEach(function(d) {
         pairRateWithId[d.id] = +d.rate;
         pairNameWithId[d.id] = d.name;
         });
        
         svg.append("g")
         .attr("class", "county")
         .selectAll("path")
         .data(topojson.feature(us, us.objects.counties).features)
         .enter().append("path")
         .attr("d", path)
         .style ( "fill" , function (d) {
         return color (pairRateWithId[d.id]);
         })
         .style("opacity", 0.8)
         .on("mouseover", function(d) {
         d3.select(this).transition().duration(300).style("opacity", 1);
         div.transition().duration(300)
         .style("opacity", 1)
         div.text(pairNameWithId[d.id] + " : " + pairRateWithId[d.id])
         .style("left", (d3.event.pageX) + "px")
         .style("top", (d3.event.pageY -30) + "px");
         })
         .on("mouseout", function() {
         d3.select(this)
         .transition().duration(300)
         .style("opacity", 0.8);
         div.transition().duration(300)
         .style("opacity", 0);
         })
    };
    var legend = svg.selectAll("g.legend")
     .data(ext_color_domain)
     .enter().append("g")
     .attr("class", "legend");

    var ls_w = 20, ls_h = 20;

    legend.append("rect")
     .attr("x", 20)
     .attr("y", function(d, i){ return height - (i*ls_h) - 2*ls_h;})
     .attr("width", ls_w)
     .attr("height", ls_h)
     .style("fill", function(d, i) { return color(d); })
     .style("opacity", 0.8);

    legend.append("text")
     .attr("x", 50)
     .attr("y", function(d, i){ return height - (i*ls_h) - ls_h - 4;})
     .text(function(d, i){ return legend_labels[i]; });
});

<IPython.core.display.Javascript object>